In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.datasets import load_sample_images

In [ ]:
images = load_sample_images()["images"]
images = keras.layers.CenterCrop(height=70, width=120)(images)
images = keras.layers.Rescaling(scale = 1 / 255)(images)

In [ ]:
images.shape # (2 sample images, 70 height, 120 width, 3 color channels)

In [ ]:
conv_layer1 = keras.layers.Conv2D(filters=32, kernel_size=(7, 7))
fmaps = conv_layer1(images)

In [ ]:
fmaps.shape # (2 images, 64 height, 114 width, 32 channels) -> shrinking because padding / "valid" is default

In [ ]:
conv_layer2 = keras.layers.Conv2D(filters=32, kernel_size=(7, 7), padding="same")
fmaps = conv_layer2(images)

In [ ]:
fmaps.shape

In [ ]:
kernels, weights = conv_layer1.get_weights()
kernels.shape, weights.shape # (kernel height, kernel width, input channels, output channels)

In [ ]:
max_pool = keras.layers.MaxPool2D(pool_size=(2, 2))
max_pool.strides

In [ ]:
pooled = max_pool(fmaps)
fmaps.shape, pooled.shape

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=(3, 3), padding="same", activation="relu", kernel_initializer="he_normal")
model = keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=(7, 7), input_shape=[28, 28, 1]),
    keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation="softmax")
])

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = X_train.reshape(60000, 28, 28, 1), X_test.reshape(10000, 28, 28, 1)
X_train.shape, X_test.shape

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
%%time
model.fit(X_train, y_train, validation_split=0.05, epochs=10, batch_size=32)

In [ ]:
%%time
model.evaluate(X_test, y_test)